# Outline
First, we will install Python using Conda.  
1. Installation.  


Then we will follow these excellent guides:  

2. [Basics](https://github.com/koldunovn/python_for_geosciences/blob/master/02%20-%20Python%20basics.ipynb) guide by Nikolay Koldunov from the course on [Python for GeoSciences](https://github.com/koldunovn/python_for_geosciences).  
3. [NumPy](https://github.com/koldunovn/python_for_geosciences/blob/master/03%20-%20NumPy%20arrays.ipynb) guide by Nikolay Koldunov from the course on [Python for GeoSciences](https://github.com/koldunovn/python_for_geosciences).  
4. [Matplotlib](https://github.com/koldunovn/python_for_geosciences/blob/master/05.1%20-%20Graphs%20(Matplotlib).ipynb) guide by Nikolay Koldunov from the course on [Python for GeoSciences](https://github.com/koldunovn/python_for_geosciences).  
5. [Cartopy](https://github.com/koldunovn/python_for_geosciences/blob/master/05.3%20-%20Maps%20(cartopy).ipynb) guide by Nikolay Koldunov from the course on [Python for GeoSciences](https://github.com/koldunovn/python_for_geosciences).  
6. [Pandas and GeoPandas](http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/geopandas.html) guide by Pangeo.  
7. [Xarray](http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/xarray.html) guide by Pangeo.  

Then we will apply these to an example WRFChem dataset:  

8. **WRFChem.**  

Advanced Python guides beyond this introduction:  

9. [Dask](http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/dask.html) guide by Pangeo.  

___

# 8. WRFChem

In [ ]:
# import the required libraries
import xarray as xr
import salem
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs

In [ ]:
# open the data set
ds = xr.open_dataset('/nfs/a68/earlacoa/shared/wrfout_d01_global_0.25deg_2015-12_PM2_5_DRY.nc')

ds

In [ ]:
# extract the data array
pm25 = ds['PM2_5_DRY']
pm25

In [ ]:
# monthly mean
pm25_mean = pm25.mean(dim='time')
pm25_mean

In [ ]:
lon = ds.lon.values
lat = ds.lat.values
    
# create 2d latitude and longitude arrays for plotting
xx, yy = np.meshgrid(lon, lat)

In [ ]:
# create a plot
fig = plt.figure(1, figsize=(8, 8))
gs = gridspec.GridSpec(1, 1)

ax = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax.set_extent([70, 140, 0, 60], crs=ccrs.PlateCarree())
ax.coastlines()
im = ax.contourf(xx, yy, pm25_mean, transform=ccrs.PlateCarree())
fig.colorbar(im, label='Ambient PM$_{2.5}$ concentrations (${\mu}g$ $m^{-3}$)', shrink=0.69)
plt.title('Dec 2015 WRFChem output')

plt.tight_layout()
plt.show()

In [ ]:
# import custom functions to cut a data array to a shapefile
from cutshapefile import transform_from_latlon, rasterize

In [ ]:
# load shapefile (single multipolygon) and extract shapes
import geopandas as gpd

shapefile = gpd.read_file('CHN_adm0.shp') # more can obtained from here: https://gadm.org/
shapes = [(shape, index) for index, shape in enumerate(shapefile.geometry)]

shapefile

In [ ]:
# apply shapefile to geometry, default: inside shapefile == 0, outside shapefile == np.nan
ds['shapefile'] = rasterize(shapes, ds.coords, longitude='lon', latitude='lat') 

In [ ]:
ds

In [ ]:
# change to more intuitive labelling of 1 for inside shapefile and np.nan for outside shapefile
# if condition preserve (outside shapefile, as inside defaults to 0), otherwise (1, to mark in shapefile)
ds['shapefile'] = ds.shapefile.where(cond=ds.shapefile!=0, other=1) 

In [ ]:
np.nan

In [ ]:
ds = ds.where(cond=ds.shapefile==1, other=np.nan) # set data outside shapefile to np.nan: if condition (in shapefile) preserve, otherwise (not in shapefile) set to np.nan
#ds = ds.where(cond=ds.shapefile!=1, other=ds*0.5) # scale data outside shapefile by 0.5: if condition (not in shapefile) preserve, otherwise (in shapefile) scale

In [ ]:
ds

In [ ]:
pm25_cropped = ds['PM2_5_DRY']

pm25_cropped_mean = pm25_cropped.mean(dim='time')
pm25_cropped_mean

In [ ]:
# create a plot
fig = plt.figure(1, figsize=(8, 8))
gs = gridspec.GridSpec(1, 1)

ax = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax.set_extent([70, 140, 0, 60], crs=ccrs.PlateCarree())
ax.coastlines()
im = ax.contourf(xx, yy, pm25_cropped_mean, transform=ccrs.PlateCarree())
fig.colorbar(im, label='Ambient PM$_{2.5}$ concentrations (${\mu}g$ $m^{-3}$)', shrink=0.69)
plt.title('Dec 2015 WRFChem output - cropped to shapefile')

plt.tight_layout()
plt.show()

In [ ]:
# open population count for 2015
pop = xr.open_dataset('/nfs/a68/earlacoa/shared/gpw_v4_population_count_rev11_2020_0.25deg_crop.nc')['pop']
pop

In [ ]:
# population-weighted exposures
pm25_cropped_popweighted = (pm25_cropped * pop) / 1_439_323_776 # population of China in 2020
pm25_cropped_popweighted_china = pm25_cropped_popweighted.sum(dim=['lat', 'lon'])
pm25_cropped_popweighted_china

In [ ]:
pm25_cropped_popweighted_china.plot()

In [ ]:
pm25_cropped_popweighted_china.mean()

In [ ]:
ds.close()